In [2]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from sklearn import *
from sklearn.metrics import *
from itertools import islice
from sklearn.preprocessing import normalize
from tensorflow.keras.models import load_model

from sklearn.model_selection import StratifiedKFold
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277

2024-03-23 16:57:46.522348: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-23 16:57:46.544274: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 16:57:46.544298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 16:57:46.544916: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-23 16:57:46.549612: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4005553798823814693
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5803868160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10051354637456163243
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2024-03-23 16:57:49.008136: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 16:57:49.026427: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 16:57:49.026463: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 16:57:49.158410: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 16:57:49.158453: I external/local_xla/xla/stream_executor

In [4]:
train_values = pd.read_csv('./data/train_test/train_patients.csv').values
test_values = pd.read_csv('./data/train_test/test_patients.csv').values
    
print(train_values.shape)
print(test_values.shape)

X_train = train_values[:,:-2]
X_test = test_values[:,:-2]

X_train = X_train.reshape(-1, X_train.shape[1], 1)
X_test = X_test.reshape(-1, X_test.shape[1], 1)

y_train = train_values[:,-2]
y_test = test_values[:,-2]

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

(38751, 277)
(72794, 277)


In [5]:
def focal_loss(gamma=2.0):
    def focal_loss_fixed(y_true, y_pred):
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        loss = -K.pow(1 - pt, gamma) * K.log(pt)
        return loss

    return focal_loss_fixed

In [7]:
lstmmodel = load_model('./models/lstmmodel.h5',custom_objects={'focal_loss_fixed': focal_loss})

2024-03-23 16:58:02.197937: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 16:58:02.198006: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 16:58:02.198022: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 16:58:02.198136: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 16:58:02.198152: I external/local_xla/xla/stream_executor

In [8]:
feature_importance_df = pd.DataFrame()

for class_num in range(1,7):

    feature_importance_df = pd.DataFrame()
    
    N_test = test_values[test_values[:,-2]==class_num]

    X_test = N_test[:,:-2]
    y_test = N_test[:len(N_test),-2]

    X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
    y_test1 = to_categorical(y_test)
    lstmpredictions = lstmmodel.predict(X_test1, verbose=1)

    lstm_predict=np.argmax(lstmpredictions,axis=1)
    lstm_actual_value=np.argmax(y_test1,axis=1)


    lstmmodel.layers[-1].activation = None

    grad_model = tf.keras.models.Model([lstmmodel.inputs], [lstmmodel.get_layer('last_lstm_layer').output, lstmmodel.output])

    with tf.GradientTape() as tape:
        last_lstm_layer_features, y_predictions = grad_model(X_test1)
    grads = tape.gradient(y_predictions, last_lstm_layer_features)

    alpha_values = tf.reduce_mean(grads, axis = 1)

    gradcam = tf.linalg.matmul(last_lstm_layer_features, alpha_values[..., tf.newaxis])


    gradcam = gradcam.numpy()
    gradcam = np.abs(gradcam)
    gradcam_norm = (gradcam - np.min(gradcam, axis = 1, keepdims = True)) / (np.max(gradcam, axis = 1, keepdims = True) - 
                                                            np.min(gradcam, axis = 1, keepdims = True))
    y_predictions = tf.nn.softmax(y_predictions)
    y_predictions = y_predictions.numpy()


    def means_of_slices(iterable, slice_size):
        iterator = iter(iterable)
        while True:
            slice = list(islice(iterator, slice_size))
            if slice:
                yield np.sum(slice)/len(slice)
            else:
                return

    cam_per_segment = []
    for i in range(X_test1.shape[0]):
        means = list(means_of_slices(gradcam_norm[i], 12))
        cam_per_segment.append(means)
    cam_per_segment = np.array(cam_per_segment)
    print(cam_per_segment.shape)


    ecg_norm = (X_test1[20, :] - X_test1[20, :].min()) / (X_test1[20, :].max() - X_test1[20, :].min())

    mean_cam_norm = cam_per_segment.mean(axis=0)

    feature_importance_normalized = (mean_cam_norm - mean_cam_norm.min()) / (mean_cam_norm.max() - mean_cam_norm.min())

    indices_sort = np.argsort(-1 * mean_cam_norm)

    slices = np.arange(1, 24)

    fig, ax = plt.subplots(1, 2, figsize = (15, 4))
    sns.barplot(x=np.arange(23), y=feature_importance_normalized[indices_sort], ax=ax[0],palette="pastel")
    ax[0].set_title('Average Grad-CAM values for LSTM model per segment over Class: ' + str(class_num))
    ax[0].set_xticks(np.arange(0,23))
    ax[0].set_xticklabels(slices[indices_sort])
    ax[0].set_xlabel('Slice')
    ax[0].set_ylabel('Average Grad-CAM values')

    ax[1].plot(np.arange(len(ecg_norm)), ecg_norm, label='ECG data')
    ax[1].plot(np.repeat(feature_importance_normalized,12), label = 'Average Grad-CAM values')
    ax[1].set_title('Average Grad-CAM values for LSTM model per segment over Class: ' + str(class_num))
    ax[1].set_xlabel('Time')
    ax[1].set_ylabel('ECG signal / Average Grad-CAM values')
    ax[1].legend()
    image_name = 'output_pics/Average Grad-CAM values for LSTM model per segment over Class: ' + str(class_num)+'.jpg'
    fig.savefig(image_name, dpi = 400)
    feature_importance_df['class_' + str(class_num)] = feature_importance_normalized

feature_importance_df.to_csv('./output_csv/GRADCAM_weights_LSTM_model.csv', index=False)


   3/1843 [..............................] - ETA: 55s  

2024-03-23 16:58:03.642864: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


 560/1843 [========>.....................] - ETA: 34s

KeyboardInterrupt: 